In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InR

In [2]:
import pandas as pd

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-02-22 23:57:00--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2023-02-22 23:57:01 (11.4 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3VR960AHLFKDV|          4|            0|          0|   N|                Y|
|R16LGVMFKIUT0G|          5|            0|          0|   N|                Y|
|R1AIMEEPYHMOE4|          5|            1|          1|   N|                Y|
|R1892CCSZWZ9SR|          3|            0|          0|   N|                Y|
|R285P679YWVKD1|          3|            0|          0|   N|                N|
| RLB33HJBXHZHU|          5|            0|          0|   N|                Y|
|R1VGTZ94DBAD6A|          5|            2|          2|   N|                Y|
|R168KF82ICSOHD|          5|            0|          0|   N|                Y|
|R20DIYIJ0OCMOG|          5|            0|          0|   N|                Y|
| RD46RNVOHNZSC|          5|            0|          0|   N|     

In [7]:
# Filter data and create new DF to get all rows where total_votes count is equal to or greater than 20. 
vote_count_df = vine_df.filter(vine_df['total_votes'] >= 20)
vote_count_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
|R3GNSIFV1J2Y2B|          1|           15|         60|   N|                N|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|
|R3OFB4P7Y8WR27|          1|           15|         26|   N|                Y|
|R3MTAYGQM25N63|          4|           58|         59|   N|     

In [8]:
# Filter the new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
helpful_votes_df = vote_count_df.filter(vote_count_df["helpful_votes"]/vote_count_df["total_votes"] >= 0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|
|R3OFB4P7Y8WR27|          1|           15|         26|   N|                Y|
|R3MTAYGQM25N63|          4|           58|         59|   N|                Y|
| RJNDSWES5ISZ7|          5|           78|         79|   N|     

In [9]:
# Filter the DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
vine_paid_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'Y')
vine_paid_df.count()

136

In [10]:
# Filter the DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'N'
vine_unpaid_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'N') 
vine_unpaid_df.count()

18019

In [11]:
# Determine the total number of reviews 
df.count()

792113

In [20]:
# Determine the number of 5-star reviews
five_star_df = df.filter(df["star_rating"] == 5)
five_star_df.count()

447716

In [22]:
# Determine the percentage of 5-star reviews for the two types of review (paid vs unpaid)
five_star_paid_df = vine_paid_df[(vine_paid_df['star_rating'] == 5)]
five_star_paid_df.count()

74

In [18]:
five_star_unpaid_df = vine_unpaid_df[(vine_unpaid_df['star_rating'] == 5)]
five_star_unpaid_df.count()

8482